# Seleccion, agrupacion y transformacion de dataframes con pandas

In [2]:
import pandas as pd
import numpy as np


In [11]:
df = pd.read_csv('datasets/NY-House-Dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4801 entries, 0 to 4800
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BROKERTITLE                  4801 non-null   object 
 1   TYPE                         4801 non-null   object 
 2   PRICE                        4801 non-null   int64  
 3   BEDS                         4801 non-null   int64  
 4   BATH                         4801 non-null   float64
 5   PROPERTYSQFT                 4801 non-null   float64
 6   ADDRESS                      4801 non-null   object 
 7   STATE                        4801 non-null   object 
 8   MAIN_ADDRESS                 4801 non-null   object 
 9   ADMINISTRATIVE_AREA_LEVEL_2  4801 non-null   object 
 10  LOCALITY                     4801 non-null   object 
 11  SUBLOCALITY                  4801 non-null   object 
 12  STREET_NAME                  4801 non-null   object 
 13  LONG_NAME         

Supongamos que quiero hacer un analisis de aquellos atributos que describen la propiedad fisica y su relacion con el precio, entonces mi seleccion de columnas podria incluir solamente 'TYPE', 'PRICE', 'BEDS', y 'BATH'. Vamos a renombrarlas y hacer un subset de esas columnas para trabajarlas

In [12]:
df.rename({'TYPE': 'tipo', 
        'PRICE': 'precio', 
        'BEDS': 'habitaciones', 
        'BATH': 'banios'}, axis=1, inplace=True)

In [13]:
# uso .loc para seleccionar las primeras 10 filas de las columnas de mi interes

df.loc[1:10, ['tipo', 'precio', 'habitaciones', 'banios']]

,tipo,precio,habitaciones,banios
1,Condo for sale,195000000,7,10.000000
2,House for sale,260000,4,2.000000
3,Condo for sale,69000,3,1.000000
4,Townhouse for sale,55000000,7,2.373861
5,House for sale,690000,5,2.000000
6,Condo for sale,899500,2,2.000000
7,House for sale,16800000,8,16.000000
8,Co-op for sale,265000,1,1.000000
9,Co-op for sale,440000,2,1.000000
10,Co-op for sale,375000,2,1.000000


In [14]:
# usamos iloc para seleccionar el rango 100 a 110, de 2 en 2
df.loc[100:110:2]

,BROKERTITLE,tipo,precio,habitaciones,banios,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
100,Brokered by Brown Harris Stevens,Co-op for sale,280000,1,1.000000,1000.000000,8015 6th Ave Apt A1,"Brooklyn, NY 11209","8015 6th Ave Apt A1Brooklyn, NY 11209",New York,Kings County,Brooklyn,Bay Ridge,8015,"8015 6th Ave Apt A1, Brooklyn, NY 11209, USA",40.624177,-74.021684
102,Built by Toll Brothers,For sale,2280000,2,2.000000,1400.000000,The Rockwell # 6E,"New York, NY 10025","The Rockwell # 6ENew York, NY 10025",United States,New York,New York County,New York,West 103rd Street,"218 W 103rd St, New York, NY 10025, USA",40.798658,-73.967904
104,Brokered by Charles Rutenberg Realty Inc,Pending,739000,6,4.000000,2184.207862,38 Dare Ct,"Brooklyn, NY 11229","38 Dare CtBrooklyn, NY 11229",United States,New York,Kings County,Brooklyn,Dare Court,"38 Dare Ct, Brooklyn, NY 11229, USA",40.587799,-73.921619
106,Brokered by Douglas Elliman - 187 Seventh Ave,House for sale,3950000,5,2.000000,3528.000000,591 4th St,"Brooklyn, NY 11215","591 4th StBrooklyn, NY 11215",United States,New York,Kings County,Brooklyn,4th Street,"591 4th St, Brooklyn, NY 11215, USA",40.668062,-73.974838
108,Brokered by Corcoran Chelsea/Flatiron,Pending,299999,3,1.000000,2184.207862,215 W 75th St Apt 4D,"New York, NY 10023","215 W 75th St Apt 4DNew York, NY 10023",New York,New York County,New York,Manhattan,215,"215 W 75th St #4d, New York, NY 10023, USA",40.781058,-73.980841
110,NoBroker,Pending,4800000,6,2.373861,3958.000000,1151 83rd St,"Brooklyn, NY 11228","1151 83rd StBrooklyn, NY 11228",United States,New York,Kings County,Brooklyn,83rd Street,"1151 83rd St, Brooklyn, NY 11228, USA",40.617818,-74.015117


## Varias maneras de hacer lo mismo.

Pandas, similar a Python, a veces cuenta con varios metodos para obtener el mismo resultado. Es cuestion de elegir segun eficiencia, elegancia, y estilo de programacion. Por lo general uno querria elegir un estilo y usarlo a lo largo de un codigo, como para que un lector externo pueda internalizar el estilo, leer y entender el codigo con mayor facilidad. 

In [15]:
# redondeamos la cantidad de baños en casos donde no hay un numero entero, y transformamos a numero entero
df['banios'] = df['banios'].round(0).astype(int)

In [16]:
# creamos una categoria para baños
def crear_categoria_banio(row):
    if row['banios'] >= 4:
        return '4 o mas baños'
    else:
        return str(row['banios'])
    
df['banios_cat'] = df[['banios']].apply(crear_categoria_banio, axis=1)
df['banios_cat'].value_counts()

banios_cat
2                1973
1                1521
4 o mas baños     837
3                 467
0                   3
Name: count, dtype: int64

In [17]:
# creamos una categoria para baños usando numpy

conditions = [
    df['banios'] >= 4,
    df['banios'] < 4
]

choices = ['4 o mas baños', df['banios'].astype(str)]

df['banios_cat'] = np.select(conditions, choices)
df['banios_cat'].value_counts()

banios_cat
2                1973
1                1521
4 o mas baños     837
3                 467
0                   3
Name: count, dtype: int64

### map en pandas

map toma el mismo nombre y significado que el metodo map para arrays en python plano. La idea es aplicar la misma funcion a una coleccion de datos, en este caso a una Series de pandas. No se puede aplicar map a un dataframe, en cambio si se puede aplicar la funcion 'apply' a un dataframe (primer ejemplo), siempre que se tenga claro el axis: si queremos aplicar sobre filas o columnas. 

In [18]:
# creamos una categoria para baños con map
df['banios_cat'] = df['banios'].map(lambda x: '4 o mas baños' if x >= 4 else str(x))
df['banios_cat'].value_counts()


banios_cat
2                1973
1                1521
4 o mas baños     837
3                 467
0                   3
Name: count, dtype: int64

In [19]:
# tambien se puede usar apply sobre una serie, y funciona igual que map
df['banios_cat'] = df['banios'].apply(lambda x: '4 o mas baños' if x >= 4 else str(x))
df['banios_cat'].value_counts()

banios_cat
2                1973
1                1521
4 o mas baños     837
3                 467
0                   3
Name: count, dtype: int64

value_counts cuenta los valores unicos de una serie y los suma, dando la cantidad de valores por cada categoria. Se puede lograr esto mismo usando el metodo groupby

In [20]:
df.groupby('banios_cat').size()

banios_cat
0                   3
1                1521
2                1973
3                 467
4 o mas baños     837
dtype: int64

Usemos la categoria nueva para agrupar las casas y obtener algunas metricas

In [21]:
# agrupar precio por categoria creada

df[['precio', 'banios_cat']].groupby('banios_cat').describe()


precio                                                   \
                count          mean           std       min        25%   
banios_cat                                                               
0                 3.0  5.448333e+06  8.267656e+06  675000.0   675000.0   
1              1521.0  4.762976e+05  3.141427e+05    2494.0   269500.0   
2              1973.0  1.683271e+06  3.627467e+06    5800.0   649000.0   
3               467.0  1.699884e+06  1.545014e+06  199999.0   862500.0   
4 o mas baños   837.0  7.717965e+06  7.467007e+07  300000.0  1295000.0   

                                                   
                     50%        75%           max  
banios_cat                                         
0               675000.0  7835000.0  1.499500e+07  
1               389888.0   599000.0  4.200000e+06  
2               868000.0  1400000.0  6.500000e+07  
3              1150000.0  1849000.0  1.249500e+07  
4 o mas baños  2200000.0  5500000.0  2.147484e+09

In [22]:
df['tipo'].value_counts()

tipo
Co-op for sale                1450
House for sale                1012
Condo for sale                 891
Multi-family home for sale     727
Townhouse for sale             299
Pending                        243
Contingent                      88
Land for sale                   49
For sale                        20
Foreclosure                     14
Condop for sale                  5
Coming Soon                      2
Mobile house for sale            1
Name: count, dtype: int64

# Ejercicios propuestos

- Obtener la cantidad de habitaciones de la propiedad con el precio mas alto
- Obtener el promedio de precios de propiedades segun el tipo
- Crear una categoria nueva en base a su criterio que resuma la columna 'tipo' a solo 5 tipos, si quieren pueden traducirlas al español. Luego vuelvan a calcular el promedio sobre esta reduccion. 